# Decision Trees Implementation


In [1]:
import numpy as np
import pandas as pd

## 1. Gini Impurity

The Cost function used to split the data.

Gini score gives you an idea how good a given split is by measuring how mixed each split actually is.

It is calculated as the following :

$G_i = 1 - \sum_{k=1}^{n}p_{i,k}^2$

where, $G_i$ is the Gini score for the $i^{th}$ node and $p_{i,k}$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node.

In [4]:
def NodesGiniScore(arrNodes):
    """
    Takes a Numpy Array containing Node's no. of class instances for each class
    & returns each Node's Gini Score
    
    arrNodes : shape = [No. of Nodes, No. of Classes], np.array type containing of Nodes with each 
                element containing the number of class instances
    
    RETURNS : 
    
    Vector containing the Gini score of each Node that was passed 
    
    Example:
    
    IN>>NodesGiniScore(np.array([[1,1,34],[1,2,3], [50,50,50]]))
    OUT>>array([0.10648148, 0.61111111, 0.66666667])
    
    
    """
    
    Nodesums = np.sum(arrNodes, axis = 1)
    
    Nodesums = Nodesums.astype(np.float64)
    
    Nodesums[Nodesums == 0] = np.inf
    
    return 1 - np.sum(np.power(arrNodes*np.reshape((1/Nodesums),[-1,1]),2), axis = 1)
    

In [5]:
NodesGiniScore(np.array([[1,1,34],[1,2,3], [50,50,50]]))

array([0.10648148, 0.61111111, 0.66666667])

In [6]:
def Split_GiniIndex(splitL, splitR, n_classes):
    """
    Calculates the Gini Index (or the Cost for the split) for the 
    split nodes entered.
    
    splitL, splitR : Arrays of shape = [No. of Rows for this split, No. of features], the array after splitting the dataset
    n_classes : The number of classes.

    NOTE: 1. Function expects for the last feature/column to contain the Class Index
             in the numerical form, i.e. like 0,1,2,...,n-1
             
          2. This function also neglects that split which is empty
    
    RETURNS:
    
    The final Gini Index for this Split.
    """
    
    
    valuesL = np.zeros(n_classes)
    valuesR = np.zeros(n_classes)
    
    if splitL.shape[0] == 0:
        for i in range(n_classes):
            valuesR[i] = np.sum(splitR[:,-1] == i)
    elif splitR.shape[0] == 0:
        for i in range(n_classes):
            valuesL[i] = np.sum(splitL[:,-1] == i)
    else:
        for i in range(n_classes):
            valuesR[i] = np.sum(splitR[:,-1] == i)
            valuesL[i] = np.sum(splitL[:,-1] == i)

    
        
    GiniL, GiniR = NodesGiniScore([valuesL, valuesR])
    
    Lrows = splitL.shape[0]
    Rrows = splitR.shape[0]
    
    GiniIndex = GiniL * Lrows/(Lrows+Rrows) + GiniR * Rrows/(Lrows+Rrows) 
    
    return GiniIndex

## 2. Splits

It is comprised of an attribute in the dataset and a value.

Splitting is done by splitting a partiular attribute/feature by a threshold value.

### 2.1. Splitting the entirety of the dataset.

This can be explained as splitting the whole dataset into two subsets via a threshold value for a particular attribute.

Once we have the two subsets, we can use Gini Score to evaluate the cost of this particular split.

Splitting a dataset involves checking if the attribute value is below or above the split value for each row in the dataset and assigning it to the left or right group respectively.

In [7]:
def Splitter(values ,threshold, feature_index = 0):
    """
    Splits the dataset (values) into two subsets via splitting through threshold
    for feature present at feature_index
    
    values : shape = [No. of rows, No. of features], The dataset to split
    threshold : Float, the value by which to compare and split the dataset
    feature_index : Integer, the index of the feature(column) through which
                    comparison will be drawn for splitting
                    
    RETURNS : 
    
    The two splits done by threshold for feature_index.
    """
    if feature_index >= values.shape[1]-1:
        raise ValueError("feature_index {} is greater the possible column index values {}. Note that it's assumed that last column is for numerical class ".format(feature_index,values.shape[1]))
    
    left = values[values[:,feature_index] <= threshold]
    right = values[values[:,feature_index] > threshold]
    
    return left, right

In [8]:
a= np.array([[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]])

In [9]:
l,r = Splitter(a, 2.771244718, 0)

In [10]:
l

array([[2.77124472, 1.78478393, 0.        ],
       [1.72857131, 1.16976141, 0.        ]])

In [11]:
r

array([[ 3.67831985,  2.81281357,  0.        ],
       [ 3.96104336,  2.61995032,  0.        ],
       [ 2.99920892,  2.20901421,  0.        ],
       [ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [12]:
Split_GiniIndex(l, r, 2)

0.375

In [13]:
NodesGiniScore([[2,0]])*(2/10) + NodesGiniScore([[3,5]])*(8/10)

array([0.375])

### 2.2. Split Evaluation

To find the best possible split, we need to check for each value for each attribute, evaluate the cost of the split and fin the best possible split.

And once the best split is found, we use that as a node in our Tree.

Therefore we try each value for each attribute in the dataset and find the best split through Gini Score.

In [87]:
def SplitEvaluator(dataset, n_classes):
    """
    Splits the dataset by each value in each attribute and then 
    finds the best split via Gini Impurity.
    
    dataset : Numpy array of shape = [No. of Rows, No. of features], the data itself in raw form.
    
    RETURNS : 
    
    Returns the Matrix containing gini Index value for the split using that row's 
    feature and threshold and index of best split in this Matrix (thus, the dataset too.)
    
    giniIndexes : Shape = [dataset.shape[0], dataset.shape[1]-1]
    (row, feat) : Coordinates for the best split in dataset or giniIndexes.
    threshold and the feature index of the threshold.
    """
    giniIndexes = np.zeros([dataset.shape[0], dataset.shape[1]-1]) #-1 to remove the class feature
    
    for feat_i in range(dataset.shape[1] - 1):
        
        for row_i in range(dataset.shape[0]):
            
            threshold = dataset[row_i,feat_i]
            
            splitL, splitR = Splitter(dataset, threshold, feat_i)
            
            #Calculate this split's Gini Index
            
            giniInd = Split_GiniIndex(splitL, splitR, n_classes)
            
            giniIndexes[row_i, feat_i] = giniInd
              
    coords =  np.argwhere(giniIndexes == np.min(giniIndexes))
        
    bestsplitL, bestsplitR = Splitter(dataset, dataset[coords[0,0], coords[0,1]], coords[0,1])
    best_gini = giniIndexes[coords[0,0], coords[0,1]]
    threshold = dataset[coords[0,0], coords[0,1]], coords[0,1]
    
    return giniIndexes, threshold, best_gini, bestsplitL, bestsplitR

#{'GiniIndex':best_gini, 'NodeL': {'data': bestsplitL, 'NodeL':None, 'NodeR':None},\
                        #'NodeR': {'data': bestsplitR, 'NodeL':None, 'NodeR':None}  } 

In [88]:
M,thres, gini, l, r = SplitEvaluator(a, 2)

In [89]:
gini

0.0

In [90]:
thres

(3.961043357, 0)

In [91]:
l

array([[2.77124472, 1.78478393, 0.        ],
       [1.72857131, 1.16976141, 0.        ],
       [3.67831985, 2.81281357, 0.        ],
       [3.96104336, 2.61995032, 0.        ],
       [2.99920892, 2.20901421, 0.        ]])

In [92]:
r

array([[ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [93]:
a

array([[ 2.77124472,  1.78478393,  0.        ],
       [ 1.72857131,  1.16976141,  0.        ],
       [ 3.67831985,  2.81281357,  0.        ],
       [ 3.96104336,  2.61995032,  0.        ],
       [ 2.99920892,  2.20901421,  0.        ],
       [ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

## 3. Tree Building

We will either have Leaf/Terminal Node or those nodes built by Recursive Splitting.

Making a Node Class

In [94]:
def NodeGinifromData(data, n_classes):
    """
    Calculates Gini Score for a Node from the data directly
    
    data : Array of shape = [n_rows, n_features]
    """
    
    values = np.zeros([n_classes])
    
    for i in range(n_classes):
        values[i] = np.sum(data[data[:,-1] == i])
        
    return NodesGiniScore([values])

In [95]:
class TreeNode:
    
    def __init__(self, data, NodeL, NodeR, n_classes):
        
        assert data.ndim == 2
        
        self.data = data
        self.values = np.zeros(n_classes)
        for i in range(n_classes):
            self.values[i] = np.sum(self.data[:,-1] == i)
        self.NodeL = NodeL
        self.NodeR = NodeR
        self.giniScore = NodeGinifromData(self.data, n_classes)[0]
        self.threshold = None
        if (((self.NodeL != None) or (self.NodeR!= None)) and self.threshold == None):
            raise ValueError("Node's Threshold not set before splitting.")
    

In [96]:
Node = TreeNode(np.array([[1,0],[24,1]]),None,None,3)

In [97]:
Node.giniScore

0.07396449704142016

### 3.1. Leaf Nodes

The Leaf Nodes will be formed only when either :

1. Maximum Tree Depth : If the next node which we add to tree exceeds this pre-defined number then we don't add this node. The problem is that deeper trees generally will overfit the data.
                        
2. Minimum Node Samples : This is the minimum number of training patterns that a given node is responsible for. Once at or below                             this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training                                 patterns are expected to be too specific and are likely to overfit the training data.

3. One-sided Split : It is possible that a split may result in one side getting all the data. If this happens then clearly it's not possible to split further, thus we stop at this too.

To make Predictions, in the leaf node, we just calculate the index of the class which has maximum entries

In [98]:
def LeafClass(LeafValues):
    """
    Returns the final class for the Leaf Node
    
    LeafValues : Array of shape = [no. of classes], No. of instances of each class
                 in the leaf node.
                 
    RETURNS:
    
    Class to which this Leaf Node belongs to.
    """
    
    return np.argmax(LeafValues)

Finally, keeping the above three conditions in mind we will start building the tree through recursive calls.

In [99]:
NodeGinifromData(a,2)

array([0.42225156])

Making a function to take input a data and find the best split and return two nodes for it

In [100]:
def TreeNodeFromNode(Node, n_classes):
    """
    Function to take input a data and find the best split and Add two nodes 
    in the Node
    
    Node : An instance of TreeNode class
    n_classes : Integer, no. of classes
    
    RETURNS :
    
    Node after adding two nodes L and R after purest split.
    """
    
    data = Node.data
    _,thres,_,dataL, dataR = SplitEvaluator(data, n_classes)
    NodeL = TreeNode(dataL, None, None, n_classes)
    NodeR = TreeNode(dataR, None, None, n_classes)
    
    Node.threshold = thres
    Node.NodeL = NodeL
    Node.NodeR = NodeR
    
    return None

In [101]:
Node = TreeNode(a, None, None, 2)

In [102]:
a

array([[ 2.77124472,  1.78478393,  0.        ],
       [ 1.72857131,  1.16976141,  0.        ],
       [ 3.67831985,  2.81281357,  0.        ],
       [ 3.96104336,  2.61995032,  0.        ],
       [ 2.99920892,  2.20901421,  0.        ],
       [ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [103]:
Node.NodeR

In [104]:
TreeNodeFromNode(Node, 2)

In [105]:
Node.data

array([[ 2.77124472,  1.78478393,  0.        ],
       [ 1.72857131,  1.16976141,  0.        ],
       [ 3.67831985,  2.81281357,  0.        ],
       [ 3.96104336,  2.61995032,  0.        ],
       [ 2.99920892,  2.20901421,  0.        ],
       [ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [106]:
Node.NodeR.data

array([[ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [107]:
Node.threshold

(3.961043357, 0)

Now a function which should be called recursively.

In [137]:
def RecursiveTreeMaker(ListOfNodes, max_depth, min_samples, curr_depth, n_classes ): 
    """
    Makes the tree recursively
    
    Let's just focus on creating a full tree which reaches max depth each time
    """
    
    
    if curr_depth > max_depth:
        return ListOfNodes
    
    newListOfNodes = []
    for node in ListOfNodes:
        
        if node.giniScore == 0.0:
            continue
            
        elif np.sum(node.values) < min_samples:
            continue 
        
        TreeNodeFromNode(node, n_classes)
        
        newListOfNodes.append(node.NodeL)
        newListOfNodes.append(node.NodeR)
        
    print(len(newListOfNodes))
    
    RecursiveTreeMaker(newListOfNodes, max_depth, min_samples, curr_depth+1, n_classes)

In [138]:
Node = TreeNode(a, None, None, 2)

In [139]:
Node.NodeL == None and Node.NodeR == None 

True

In [140]:
Node.data

array([[ 2.77124472,  1.78478393,  0.        ],
       [ 1.72857131,  1.16976141,  0.        ],
       [ 3.67831985,  2.81281357,  0.        ],
       [ 3.96104336,  2.61995032,  0.        ],
       [ 2.99920892,  2.20901421,  0.        ],
       [ 7.49754587,  3.16295355,  1.        ],
       [ 9.00220326,  3.33904719,  1.        ],
       [ 7.44454233,  0.47668338,  1.        ],
       [10.12493903,  3.23455098,  1.        ],
       [ 6.64228735,  3.31998376,  1.        ]])

In [141]:
Node.giniScore

0.42225155781074575

In [146]:
RecursiveTreeMaker([Node], 3,5,0, 2)

2
0
0
0


In [147]:
Node.NodeR.values

array([0., 5.])

In [148]:
def PrintTree(Node):
    
    
    
    print("Head")
    print('\n')
    print("Data\n")
    print(Node.data)
    print('\n')
    print("Value: ", Node.values)
    print('\n')
    print("GiniScore: ", Node.giniScore)
    
   
    
    if Node.NodeL != None:
    
        print("\n\n\n")
        print("Left")
        print('\n')
        print("Data\n")
        print(Node.NodeL.data)
        print('\n')
        print("Value: ", Node.NodeL.values)
        print('\n')
        print("GiniScore: ", Node.NodeL.giniScore)
        
    if Node.NodeR != None:

        print("\n\n\n")
        print("Right")
        print('\n')
        print("Data\n")
        print(Node.NodeR.data)
        print('\n')
        print("Value: ", Node.NodeR.values)
        print('\n')
        print("GiniScore: ", Node.NodeR.giniScore)
    
    
    elif ((Node.NodeL == None)&(Node.NodeR == None)):
        return

In [149]:
PrintTree(Node)

Head


Data

[[ 2.77124472  1.78478393  0.        ]
 [ 1.72857131  1.16976141  0.        ]
 [ 3.67831985  2.81281357  0.        ]
 [ 3.96104336  2.61995032  0.        ]
 [ 2.99920892  2.20901421  0.        ]
 [ 7.49754587  3.16295355  1.        ]
 [ 9.00220326  3.33904719  1.        ]
 [ 7.44454233  0.47668338  1.        ]
 [10.12493903  3.23455098  1.        ]
 [ 6.64228735  3.31998376  1.        ]]


Value:  [5. 5.]


GiniScore:  0.42225155781074575




Left


Data

[[2.77124472 1.78478393 0.        ]
 [1.72857131 1.16976141 0.        ]
 [3.67831985 2.81281357 0.        ]
 [3.96104336 2.61995032 0.        ]
 [2.99920892 2.20901421 0.        ]]


Value:  [5. 0.]


GiniScore:  0.0




Right


Data

[[ 7.49754587  3.16295355  1.        ]
 [ 9.00220326  3.33904719  1.        ]
 [ 7.44454233  0.47668338  1.        ]
 [10.12493903  3.23455098  1.        ]
 [ 6.64228735  3.31998376  1.        ]]


Value:  [0. 5.]


GiniScore:  0.0


In [150]:
sample_data = np.random.normal(6,3,[100,4])

In [151]:
sample_data[:5,:]

array([[ 4.24351657,  9.09206202,  9.09792826,  5.41717701],
       [ 9.71431478,  3.57255848,  7.51859789, 10.07536702],
       [ 3.38600403,  9.71676306,  7.91752294,  9.44748285],
       [ 7.84640874,  1.48088124,  7.44012912,  3.53729783],
       [ 8.39551102, -0.7959244 ,  3.58928619,  5.48416522]])

In [152]:
sample_data = np.c_[sample_data, np.zeros([100,1])]

In [153]:
sample_data[60:,-1] = 1

In [154]:
sample_data

array([[ 4.24351657e+00,  9.09206202e+00,  9.09792826e+00,
         5.41717701e+00,  0.00000000e+00],
       [ 9.71431478e+00,  3.57255848e+00,  7.51859789e+00,
         1.00753670e+01,  0.00000000e+00],
       [ 3.38600403e+00,  9.71676306e+00,  7.91752294e+00,
         9.44748285e+00,  0.00000000e+00],
       [ 7.84640874e+00,  1.48088124e+00,  7.44012912e+00,
         3.53729783e+00,  0.00000000e+00],
       [ 8.39551102e+00, -7.95924401e-01,  3.58928619e+00,
         5.48416522e+00,  0.00000000e+00],
       [ 6.26296227e+00,  1.83439813e+00,  8.20088745e+00,
         4.50917842e+00,  0.00000000e+00],
       [ 9.41507834e+00,  3.70021566e+00,  8.55579883e+00,
         7.34419750e+00,  0.00000000e+00],
       [ 8.27348898e+00,  6.71061870e+00,  4.11947831e+00,
         6.42477164e+00,  0.00000000e+00],
       [ 5.77219544e+00,  2.27096013e+00, -7.71409089e-03,
         8.09106565e+00,  0.00000000e+00],
       [ 8.46791860e+00,  1.12990704e+01,  4.71328204e+00,
         5.88580017e+00

In [155]:
Root = TreeNode(sample_data, None, None, 2)

In [156]:
RecursiveTreeMaker([Root], 4, 10, 0, 2)

2
4
4
4
2


In [166]:
def TreeMaker(dataset, max_depth, min_samples, n_classes):
    """
    Creates the Tree and returns it's Root Node
    
    dataset : Array of shape = [n_rows, n_feat]
    max_depth : Maximum depth of the tree
    n_classes : No.of classes
    """
    
    Root = TreeNode(dataset, None, None, n_classes)
    
    RecursiveTreeMaker([Root], max_depth, min_samples, 1 , n_classes)
    
    return Root

In [168]:
Root = TreeMaker(sample_data, 5, 10, 2)

2
4
4
4
2


In [171]:
class DecisionTrees:

    
    """
    ***************DECISION TREES********************
    
    Call function TreeMaker() to get the Root of the tree with all connections to 
    next branches.
    """
     
    def __init__(self, dataset, max_depth, min_samples, n_classes):
        
        self.dataset = dataset
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.n_classes = n_classes
        
        
    
    class TreeNode:
    
        def __init__(self, data, NodeL, NodeR, n_classes):

            assert data.ndim == 2

            self.data = data
            self.values = np.zeros(n_classes)
            for i in range(n_classes):
                self.values[i] = np.sum(self.data[:,-1] == i)
            self.NodeL = NodeL
            self.NodeR = NodeR
            self.giniScore = NodeGinifromData(self.data, n_classes)[0]
            self.threshold = None
            if (((self.NodeL != None) or (self.NodeR!= None)) and self.threshold == None):
                raise ValueError("Node's Threshold not set before splitting.")
    
    
    def NodesGiniScore(arrNodes):
        """
        Takes a Numpy Array containing Node's no. of class instances for each class
        & returns each Node's Gini Score

        arrNodes : shape = [No. of Nodes, No. of Classes], np.array type containing of Nodes with each 
                    element containing the number of class instances

        RETURNS : 

        Vector containing the Gini score of each Node that was passed 

        Example:

        IN>>NodesGiniScore(np.array([[1,1,34],[1,2,3], [50,50,50]]))
        OUT>>array([0.10648148, 0.61111111, 0.66666667])


        """

        Nodesums = np.sum(arrNodes, axis = 1)

        Nodesums = Nodesums.astype(np.float64)

        Nodesums[Nodesums == 0] = np.inf

        return 1 - np.sum(np.power(arrNodes*np.reshape((1/Nodesums),[-1,1]),2), axis = 1)
    
    
    def Split_GiniIndex(splitL, splitR, n_classes):
        """
        Calculates the Gini Index (or the Cost for the split) for the 
        split nodes entered.

        splitL, splitR : Arrays of shape = [No. of Rows for this split, No. of features], the array after splitting the dataset
        n_classes : The number of classes.

        NOTE: 1. Function expects for the last feature/column to contain the Class Index
                 in the numerical form, i.e. like 0,1,2,...,n-1

              2. This function also neglects that split which is empty

        RETURNS:

        The final Gini Index for this Split.
        """


        valuesL = np.zeros(n_classes)
        valuesR = np.zeros(n_classes)

        if splitL.shape[0] == 0:
            for i in range(n_classes):
                valuesR[i] = np.sum(splitR[:,-1] == i)
        elif splitR.shape[0] == 0:
            for i in range(n_classes):
                valuesL[i] = np.sum(splitL[:,-1] == i)
        else:
            for i in range(n_classes):
                valuesR[i] = np.sum(splitR[:,-1] == i)
                valuesL[i] = np.sum(splitL[:,-1] == i)



        GiniL, GiniR = NodesGiniScore([valuesL, valuesR])

        Lrows = splitL.shape[0]
        Rrows = splitR.shape[0]

        GiniIndex = GiniL * Lrows/(Lrows+Rrows) + GiniR * Rrows/(Lrows+Rrows) 
    
        return GiniIndex
    

    def Splitter(values ,threshold, feature_index = 0):
        """
        Splits the dataset (values) into two subsets via splitting through threshold
        for feature present at feature_index

        values : shape = [No. of rows, No. of features], The dataset to split
        threshold : Float, the value by which to compare and split the dataset
        feature_index : Integer, the index of the feature(column) through which
                        comparison will be drawn for splitting

        RETURNS : 

        The two splits done by threshold for feature_index.
        """
        if feature_index >= values.shape[1]-1:
            raise ValueError("feature_index {} is greater the possible column index values {}. Note that it's assumed that last column is for numerical class ".format(feature_index,values.shape[1]))

        left = values[values[:,feature_index] <= threshold]
        right = values[values[:,feature_index] > threshold]

        return left, right
    
    
    def SplitEvaluator(dataset, n_classes):
        """
        Splits the dataset by each value in each attribute and then 
        finds the best split via Gini Impurity.

        dataset : Numpy array of shape = [No. of Rows, No. of features], the data itself in raw form.

        RETURNS : 

        Returns the Matrix containing gini Index value for the split using that row's 
        feature and threshold and index of best split in this Matrix (thus, the dataset too.)

        giniIndexes : Shape = [dataset.shape[0], dataset.shape[1]-1]
        (row, feat) : Coordinates for the best split in dataset or giniIndexes.
        threshold and the feature index of the threshold.
        """
        giniIndexes = np.zeros([dataset.shape[0], dataset.shape[1]-1]) #-1 to remove the class feature

        for feat_i in range(dataset.shape[1] - 1):

            for row_i in range(dataset.shape[0]):

                threshold = dataset[row_i,feat_i]

                splitL, splitR = Splitter(dataset, threshold, feat_i)

                #Calculate this split's Gini Index

                giniInd = Split_GiniIndex(splitL, splitR, n_classes)

                giniIndexes[row_i, feat_i] = giniInd

        coords =  np.argwhere(giniIndexes == np.min(giniIndexes))

        bestsplitL, bestsplitR = Splitter(dataset, dataset[coords[0,0], coords[0,1]], coords[0,1])
        best_gini = giniIndexes[coords[0,0], coords[0,1]]
        threshold = dataset[coords[0,0], coords[0,1]], coords[0,1]

        return giniIndexes, threshold, best_gini, bestsplitL, bestsplitR

#{'GiniIndex':best_gini, 'NodeL': {'data': bestsplitL, 'NodeL':None, 'NodeR':None},\
                        #'NodeR': {'data': bestsplitR, 'NodeL':None, 'NodeR':None}  } 
    
    def NodeGinifromData(data, n_classes):
        
        """
        Calculates Gini Score for a Node from the data directly

        data : Array of shape = [n_rows, n_features]
        """

        values = np.zeros([n_classes])

        for i in range(n_classes):
            values[i] = np.sum(data[data[:,-1] == i])

        return NodesGiniScore([values])
    
    
    def LeafClass(LeafValues):
        """
        Returns the final class for the Leaf Node

        LeafValues : Array of shape = [no. of classes], No. of instances of each class
                     in the leaf node.

        RETURNS:

        Class to which this Leaf Node belongs to.
        """

        return np.argmax(LeafValues)
    
    def TreeNodeFromNode(Node, n_classes):
        """
        Function to take input a data and find the best split and Add two nodes 
        in the Node

        Node : An instance of TreeNode class
        n_classes : Integer, no. of classes

        RETURNS :

        Node after adding two nodes L and R after purest split.
        """

        data = Node.data
        _,thres,_,dataL, dataR = SplitEvaluator(data, n_classes)
        NodeL = TreeNode(dataL, None, None, n_classes)
        NodeR = TreeNode(dataR, None, None, n_classes)

        Node.threshold = thres
        Node.NodeL = NodeL
        Node.NodeR = NodeR

        return None
    
        
    def RecursiveTreeMaker(ListOfNodes, max_depth, min_samples, curr_depth, n_classes ): 
        """
        Makes the tree recursively
        
        """


        if curr_depth > max_depth:
            return ListOfNodes

        newListOfNodes = []
        for node in ListOfNodes:

            if node.giniScore == 0.0:
                continue

            elif np.sum(node.values) < min_samples:
                continue 

            TreeNodeFromNode(node, n_classes)

            newListOfNodes.append(node.NodeL)
            newListOfNodes.append(node.NodeR)

        print(len(newListOfNodes))

        RecursiveTreeMaker(newListOfNodes, max_depth, min_samples, curr_depth+1, n_classes)

        
    def TreeMaker(self):
        """
        Creates the Tree and returns it's Root Node

        dataset : Array of shape = [n_rows, n_feat]
        max_depth : Maximum depth of the tree
        min_samples : Minimum no. of samples to be present at a Node,
                        otherwise node wouldn't be constructed.
        n_classes : No.of classes
        """

        Root = TreeNode(self.dataset, None, None, self.n_classes)

        RecursiveTreeMaker([Root], self.max_depth, self.min_samples, 1 , self.n_classes)

        return Root  


In [172]:
Tree = DecisionTrees(sample_data, max_depth=5, min_samples=10, n_classes=2)

In [174]:
Root = Tree.TreeMaker()

2
4
4
4
2
